- RAGAS overview: https://gist.github.com/donbr/1a1281f647419aaacb8673223b69569c
- https://github.com/explodinggradients/ragas/blob/main/docs/getstarted/rag_testset_generation.md
- None-English: https://docs.ragas.io/en/stable/howtos/customizations/testgenerator/_language_adaptation

In [2]:
pip install -qU --no-cache-dir langchain-openai==0.3.28 langchain-community==0.3.27 ragas==0.3.0 python-dotenv==1.1.1 unstructured[md]==0.18.5 pillow==10.4.0

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [1]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import os

generator_llm = LangchainLLMWrapper(
    ChatOpenAI(
        model=os.getenv("GENERATOR_MODEL"),
        api_key=os.getenv("GENERATOR_API_KEY"),
        base_url=os.getenv("GENERATOR_BASE_URL")
    )
)

generator_embeddings = LangchainEmbeddingsWrapper(
    OpenAIEmbeddings(
        model=os.getenv("EMBEDDER_MODEL"),
        api_key=os.getenv("EMBEDDER_API_KEY"),
        base_url=os.getenv("EMBEDDER_BASE_URL")
    )
)

/media/limcheekin/My Passport/ws/py/rag-eval-data-gen/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_community.document_loaders import DirectoryLoader

path = "docs/"
loader = DirectoryLoader(path, glob="**/*.md")
docs = loader.load()

In [4]:
%%script echo 'skipping this cell'

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_core.documents import Document

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"), 
    ("###", "Header 3"),
    ("####", "Header 4"),
    ("#####", "Header 5"),
    ("######", "Header 6"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers=False,  # Keep headers in content for context
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    add_start_index=True,
)
print("✅ Text Chunkers initialized.")

print("\n🔄 Chunking documents...")
md_split_docs = []
for doc in docs:
    md_header_splits = markdown_splitter.split_text(doc.page_content)
    md_header_splits = text_splitter.split_documents(md_header_splits)
    # Convert back to Document objects, preserving original metadata
    for split_chunk in md_header_splits:
        headings_list = []
        # Extract header values in order based on headers_to_split_on
        for _, header_meta_key_name in headers_to_split_on:
            if header_meta_key_name in split_chunk.metadata:
                headings_list.append(split_chunk.metadata[header_meta_key_name])

        md_split_docs.append(Document(
            page_content=split_chunk.page_content,
            metadata={**doc.metadata, "headings": headings_list}
        ))
print(f"👍 Documents split into {len(md_split_docs)} chunks.")
docs = md_split_docs

skipping this cell


In [5]:
len(docs)

1

In [6]:
docs[0]

Document(metadata={'source': 'docs/AI-奇门遁甲-02.md'}, page_content='《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精华\n\n第一章\n\n易学的科学价值主要在思维科学\n\n1. 易学的历史发展\n\n起源与演变：从伏羲画八卦开始，历经夏《连山》、商《归藏》、周《周易》的成型，到孔子作"十翼"（易传），汉代列为群经之首，形成完整的易学体系。\n\n流派分化：清代《四库全书》概括易学为"两派六宗"------象数派（含象数、机祥、图书）与义理派（含儒理、考史），两者长期争论并存，涵盖哲学、天文、军事等广泛领域。\n\n2. 易学的现代研究\n\n全球影响：近代以来，易学从中国走向世界，引发跨学科研究热潮，包括社会科学、自然科学的多元视角。\n\n科学评价：中外学者（如邓拓、荣格、张协和）高度评价《周易》的智慧价值，认为其宇宙观与规律性研究对现代科学（如量子物理）有启发，甚至与诺贝尔奖成果相关。\n\n3. 易学的科学价值争议\n\n观点分歧：\n\n部分人认为价值仅在"易理"（哲学），否定"象数/数术"的科学性；\n\n另一派认为"象数"是易理的应用体现，两者不可割裂；\n\n折中观点主张"学"（理论）与"术"（应用）结合，古为今用。\n\n作者立场：支持第三种观点，强调象、数、理、占是统一整体，易学价值需从理论和应用两方面综合研究。\n\n4. 易学的科学定位\n\n思维科学为核心：作者引用钱学森的现代科学体系分类，提出易学应归入思维科学，因其提供了独特的认知模式（如辩证、象数思维），对自然科学、社会科学及人生实践均有指导意义。\n\n实践价值：易学不仅包含哲学思想（基础科学层次），也通过数术技术（工程技术层次）联系实际，促进物质与精神文明发展。\n\n5. 结论\n\n易学作为中华文化的智慧总汇，其核心价值在于思维科学领域，兼具理论深度与实践意义，需以开放态度继承发展，服务于现代科学与文明建设。\n\n第二章\n\n人脑思维方式与易学的产生\n\n1. 人类思维方式的分类\n\n现代思维科学将人类思维方式分为三类：\n\n逻辑思维（理性思维）：代表学科如数学、物理、化学。\n\n形象思维（感性思维）：代表学科如文学、艺术、音乐。\n\n灵感思维（感应思维/直觉

## Adapt prompts to generate data in Chinese

### For single document
This is the default, using multiple documents option below for more than 1 document

In [ ]:
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)


query_distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 1.0),
]

for query, _ in query_distribution:
    prompts = await query.adapt_prompts("chinese", llm=generator_llm)
    query.set_prompts(**prompts)

### For multiple documents

In [21]:
%%script echo 'skipping this cell'
from ragas.testset.synthesizers import default_query_distribution

query_distribution = default_query_distribution(generator_llm)

for query, _ in query_distribution:
    prompts = await query.adapt_prompts("chinese", llm=generator_llm)
    query.set_prompts(**prompts)  

print(query_distribution)

skipping this cell


In [8]:
from ragas.testset.persona import Persona

personas = [
    Persona(name="学生", role_description="刚接触这个主题，需要基础解释"),
    Persona(name="专家", role_description="熟悉内容，关注深入问题"),
]

In [9]:
from ragas.testset.transforms.extractors.llm_based import NERExtractor
from ragas.testset.transforms.splitters import HeadlineSplitter

transforms = [HeadlineSplitter(), NERExtractor(llm=generator_llm)]

In [10]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType

kg = KnowledgeGraph()

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 1, relationships: 0)

In [ ]:
from ragas.testset.transforms import apply_transforms
apply_transforms(kg, transforms)

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]unable to apply transformation: 'headlines' property not found in this node


KnowledgeGraph(nodes: 1, relationships: 0)

In [18]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, 
                             embedding_model=generator_embeddings, 
                             persona_list=personas, 
                             knowledge_graph=kg)
dataset = generator.generate(
            testset_size=10, 
            query_distribution=query_distribution
        )

Generating Samples: 100%|██████████| 10/10 [00:08<00:00,  1.15it/s]


In [19]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"How does China's ancient wisdom, specifically ...",[《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精华...,"According to the text, Yi Xue, originating in ...",single_hop_specifc_query_synthesizer
1,根据这段文字，荣格对《周易》的评价是什么？,[《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精华...,根据这段文字，荣格高度评价《周易》的智慧价值。,single_hop_specifc_query_synthesizer
2,中华文化中的易学，它对现代科学有什么启发？,[《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精华...,易学从中国走向世界，引发跨学科研究热潮，包括社会科学、自然科学的多元视角。中外学者高度评价《...,single_hop_specifc_query_synthesizer
3,汉代易学的主要特点是什么？,[《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精华...,"汉代易学的一个特点是象数易学与五行学说的极端化。董仲舒提出""天人感应""，将自然现象与人事吉凶...",single_hop_specifc_query_synthesizer
4,中华文化中的易学，其科学价值主要体现在哪个方面？,[《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精华...,易学作为中华文化的智慧总汇，其核心价值在于思维科学领域，兼具理论深度与实践意义，需以开放态度...,single_hop_specifc_query_synthesizer
5,作为学生，我想了解一下易学从中国走向世界后，对全球产生了哪些影响？,[《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精华...,近代以来，易学从中国走向世界，引发跨学科研究热潮，包括社会科学、自然科学的多元视角。,single_hop_specifc_query_synthesizer
6,请问《神奇之门》这本书里提到的诺贝尔奖和易学有什么关系？,[《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精华...,《神奇之门》这本书中提到，中外学者高度评价《周易》的智慧价值，认为其宇宙观与规律性研究对现代...,single_hop_specifc_query_synthesizer
7,根据你对易学现代研究的了解，张协和对《周易》的评价是什么？,[《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精华...,张协和高度评价《周易》的智慧价值，认为其宇宙观与规律性研究对现代科学有启发，甚至与诺贝尔奖成...,single_hop_specifc_query_synthesizer
8,请问根据《神奇之门: 奇门遁甲大解谜》这本书，肿华文化的核心价值是什么？,[《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精华...,根据《神奇之门: 奇门遁甲大解谜》这本书，中华文化的核心价值在于思维科学领域，兼具理论深度与...,single_hop_specifc_query_synthesizer
9,谁撰写了《易传》，并且它对孔子研究《周易》有何影响？,[《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精华...,孔子撰写了《易传》，提升了《周易》的哲学与社会价值。孔子晚年深入研究《周易》，撰写《易传》。,single_hop_specifc_query_synthesizer


In [20]:
eval_dataset = dataset.to_evaluation_dataset()
print("Query:", eval_dataset[0].user_input)
print("Reference:", eval_dataset[0].reference)

Query: How does China's ancient wisdom, specifically Yi Xue, relate to modern scientific thought, and are there specific examples of this connection?
Reference: According to the text, Yi Xue, originating in China, is considered a treasure trove of Chinese culture with its core value residing in the field of thinking science. It offers unique cognitive models like dialectical and象数思维, which provide guidance for natural science, social science, and life practice. Some scholars believe 《周易》's wisdom and its research on the universe and its laws are insightful for modern science, such as quantum physics, and even related to Nobel Prize-winning achievements. The阴阳爻 are similar to the computer's binary code of 0 and 1, showcasing highly abstract symbolic thinking. The text also mentions that the易学符号系统, though originating from intuitive analogy, has structural laws that coincide with later scientific discoveries like genetics and mathematics, suggesting it contains model thinking that transce

In [ ]:
dataset.to_pandas().to_csv('data.csv', encoding='utf-8-sig', index=False)

In [23]:
import pandas as pd

df = pd.read_csv('data.csv', encoding='utf-8-sig')
df.head()

,user_input,reference_contexts,reference,synthesizer_name
0,"How does China's ancient wisdom, specifically ...",['《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精...,"According to the text, Yi Xue, originating in ...",single_hop_specifc_query_synthesizer
1,根据这段文字，荣格对《周易》的评价是什么？,['《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精...,根据这段文字，荣格高度评价《周易》的智慧价值。,single_hop_specifc_query_synthesizer
2,中华文化中的易学，它对现代科学有什么启发？,['《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精...,易学从中国走向世界，引发跨学科研究热潮，包括社会科学、自然科学的多元视角。中外学者高度评价《...,single_hop_specifc_query_synthesizer
3,汉代易学的主要特点是什么？,['《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精...,"汉代易学的一个特点是象数易学与五行学说的极端化。董仲舒提出""天人感应""，将自然现象与人事吉凶...",single_hop_specifc_query_synthesizer
4,中华文化中的易学，其科学价值主要体现在哪个方面？,['《神奇之门: 奇门遁甲大解谜》\n\n张志春著\n\n上 编\n\n易学思维中的科学性精...,易学作为中华文化的智慧总汇，其核心价值在于思维科学领域，兼具理论深度与实践意义，需以开放态度...,single_hop_specifc_query_synthesizer
